# Latex Headers

 - Ensure proper support for certain latex notation
 - Code below

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$
$$\newcommand{\adagger}[0]{\hat{a}^{\dagger}}$$
$$\newcommand{\ahat}[0]{\hat{a}}$$
$$\newcommand{\bdagger}[0]{\hat{b}^{\dagger}}$$
$$\newcommand{\bhat}[0]{\hat{b}}$$
$$\newcommand{\cdagger}[0]{\hat{c}^{\dagger}}$$
$$\newcommand{\chat}[0]{\hat{c}}$$
$$\newcommand{\ddagger}[0]{\hat{d}^{\dagger}}$$
$$\newcommand{\dhat}[0]{\hat{d}}$$
$$\newcommand{\edagger}[0]{\hat{e}^{\dagger}}$$
$$\newcommand{\ehat}[0]{\hat{e}}$$
$$\newcommand{\fdagger}[0]{\hat{f}^{\dagger}}$$
$$\newcommand{\fhat}[0]{\hat{f}}$$
$$\newcommand{\gtwo}[0]{g^{(2)}}$$

# SPDC Simulation by QuTip
We can model the SPDC process and squeezed light via the formalism of the degenerate parametric amplifier.

In [ ]:
!pip install qutip

from qutip import *

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
from numpy import *
# install QuTip



### Operators and Hamiltonian

In [12]:
7+9

16

In the following code, we set the parameters chi, N1, and N2, which are used to define the system's operators a, na, b, and nb. These operators represent the annihilation and number operators for two distinct modes of the system.

The tensor() function is used to create composite operators that act on the joint Hilbert space of the two modes. The destroy() and num() functions create the annihilation and number operators, respectively.

We obtain the interaction Hamiltonian Hab from the expection of the pump laser field $$| \alpha_p\rangle = D(\alpha_p) |0\rangle $$ where the displacement operator $$D(\alpha_p)$$ creates the coherent state in mode p. Let's assume  $$\alpha_p \in \mathbb{R} $$ to avoid some math tedium. Then, in the interaction picture,

$$
\begin{aligned}
H_{ab} &= \langle \alpha_p | H_{abp} | \alpha_p \rangle \\
&= -i \chi_2 (a b \alpha_p^* - a^{\dagger} b^{\dagger} \alpha_p) \\
&= -i \chi_2 \alpha_p (a b - a^{\dagger} b^{\dagger}) \\
&= -i \hbar \frac{\chi}{2} (a b - a^{\dagger} b^{\dagger})
\end{aligned}
$$

where we define

$$
\chi \equiv \frac{2 \chi_2 |\alpha_p|}{\hbar}.
$$

Here, we explicitly factored out the $\hbar$ term to simplify the subsequent math since that $\hbar$ will be canceled by the $\hbar$ in the denominator of the time dependent Schrodinger equation, $\frac{\partial}{\partial t} |\psi(t)\rangle = -\frac{i}{\hbar} H |\psi(t)\rangle$.

By default, the unit of time in Qutip is set to dimensionless, so $\hbar$ is used to convert between dimensionless time and physical time.
 


In [ ]:
#Constants
hbar=6.67e-34/(2*pi)
eps0=8.85e-12
c=3e8
#Parameters
phase_matching_bandwidth=1e13 # approximate in Hz
n_refr = 2.2
beam_power=1e-2 # in Watts
beam_area=1e-5**2 # in m^2
pump_intensity=beam_power/beam_area
alpha_p = pow(2*pump_intensity/(c*eps0*n_refr),1/2)

#presentation parameters
plot_fontsize=12

#print(alpha_p)
chi2=2.7e-12 # (2.7 pm/V according to https://www.unitedcrystals.com/BBOProp.html) 
#chi = 0.2
chi = 2*chi2*alpha_p/hbar
print(chi*hbar)
N1 = 10
N2 = 10


# definitions of operators 
a  = tensor(destroy(N1), qeye(N2))
na = tensor(num(N1),     qeye(N2)) 
b  = tensor(qeye(N1),    destroy(N2))
nb = tensor(qeye(N1),    num(N2)) 

#We obtain the following interaction Hamiltonian on the three fields $$a,b,p$$ after tracing out the pump field 
# the fully quantum 3-wave mixing Hamiltonion would be H_abp = chi2 * (a * b * p.dag() + a.dag() * b.dag() * p)

H0 = 0*a # interaction Hamiltonian before the 3-wave mixing (used for calculating initial state)
Hab = - 1j* hbar *chi * (a * b - a.dag() * b.dag())/2



### Initial state and state evolution

In the following, we'll will: 
* initialize the input state psi0 of modes a and b, which are both in the vacuum state 
* tlist: tlist is a list of times that specifies the times at which the system should be evaluated during the evolution. In the example, tlist is created using the numpy function linspace, which generates an array of evenly spaced values over a specified interval. 

* c_ops is a list of collapse operators that describe the sources of decoherence and dissipation in the system. Collapse operators represent the interactions of the system with its environment that cause transitions between the energy eigenstates of the system.
* e_ops is a python list storing observables. Here, the e_ops list is empty, which means that there are no observables being monitored during the evolution of the system.

* We use the function mesolve from the Qutip library to solve the [quantum master equation ](https://https://en.wikipedia.org/wiki/Quantum_master_equation) to find the time evolution of the system. The function takes as input the Hamiltonian Hab, the initial state psi0, the list of times tlist, the list of collapse operators c_ops, and the list of observables e_ops. The function returns an object output that contains information about the time evolution of the system, such as the states of the system at each time in tlist.



In the example, the c_ops list is empty, which means that there are no sources of decoherence or dissipation in the system. As a result, the evolution of the system is purely unitary and described by the Hamiltonian $H_{ab}$. 

The following code uses a python dictionary data structure to conveniently store operators and expected values. In anticipation of correlation values to be considered below, we're including those in our dictionary as well.  

In [ ]:
# start in the ground (vacuum) state
psi0 = tensor(basis(N1,0), basis(N2,0)) # initial state in the vacuum
# REVISION: to allow for some background light contamination (room lights, fluorescence, etc), let's actually start in a thermal state with mean photon number..
# REVISION2: with thermal state, the code has to do density matrix evolution which is super slow. let's approximate by equally bright coherent state (which is an equally good approximation as there may be some laser background etc)
#rho0= tensor(thermal_dm(N1, background_photons),thermal_dm(N2, background_photons))
#psi0= tensor(displace(N1,sqrt(background_photons)),displace(N2,sqrt(background_photons)))*psi0
#psi0= tensor(displace(N1,sqrt(background_photons)),displace(N2,sqrt(background_photons)))*psi0
theta=0
background_photons=1e-20
#c_ops = [] # c_ops is a list of collapse operators that describe the sources of decoherence and dissipation in the system. Collapse operators represent the interactions of the system with its environment that cause transitions between the energy eigenstates of the system.
n_th = background_photons  # bath temperature in terms of excitation number
c_ops = []# [sqrt((1+n_th)) * a, sqrt(n_th) * a.dag()]

psi0=cos(theta)**2*psi0 +sin(theta)**2 * tensor(displace(N1,sqrt(background_photons)),qeye(N2))*psi0
#rho0 = steadystate(H0, c_ops)



tlist = np.linspace(0, 1000, 1000) # tlist is a list of times that specifies the times at which the system should be evaluated during the evolution. In the example, tlist is created using the numpy function linspace, which generates an array of evenly spaced values over a specified interval.

e_ops = [] # e_ops is a python list storing observables. Here, the e_ops list is empty, which means that there are no observables being monitored during the evolution of the system
psi_out = mesolve(Hab, psi0, tlist, c_ops, e_ops)

ops_dict = {
  "na": a.dag() * a,
  "na_squared": (a.dag() * a)**2,
  "nb": b.dag() * b,
  "nb_squared": (b.dag() * b)**2,
}

expected_vals_dict = {key: np.zeros(len(psi_out.states)) for key in ops_dict.keys()}

for idx, psi in enumerate(psi_out.states):
  for key in ops_dict.keys():
    expected_vals_dict[key][idx] = expect(ops_dict[key], psi)
    

Let's study $|\psi_{out} \rangle$ by plotting four subplots that show the average number of photons in each mode a and b, as well as their fluctuations, as a function of time. The subplots are labeled with the corresponding quantities and the x-axis of all subplots is shared.

In [ ]:
fig, axes = plt.subplots(2, 2, sharex=True, figsize=(12,8))

tlistp=tlist[1:]
lines = [
    [axes[0,0].plot(tlist, expected_vals_dict["na"], 'rx', linewidth=2), r'$\langle \psi_{out}| n_a| \psi_{out} \rangle \equiv \langle \psi_{out}| a^\dagger a | \psi_{out} \rangle \equiv \langle a^\dagger a \rangle$'],
    [axes[0,1].plot(tlist, expected_vals_dict["nb"], 'bx', linewidth=2), r'$\langle n_b \rangle$'],
    [axes[1,0].plot(tlist, np.sqrt(expected_vals_dict["na_squared"] - expected_vals_dict["na"]**2), 'rx', linewidth=2), r'$\sqrt{\langle( n_a - \langle n_a \rangle )^{2} \rangle }$'],
    [axes[1,1].plot(tlist, np.sqrt(expected_vals_dict["nb_squared"] - expected_vals_dict["nb"]**2), 'bx', linewidth=2), r'$\sqrt{\langle( n_b -\langle n_b \rangle)^{2}\rangle }$'],
    [axes[1,0].plot(tlist, np.sqrt(expected_vals_dict["na"]**2+expected_vals_dict["na"]), 'k-', linewidth=2), r'$\sqrt{\langle n_a \rangle + \langle n_a \rangle^{2} }$'],
    [axes[1,1].plot(tlist, np.sqrt(expected_vals_dict["nb_squared"]), 'k-', linewidth=2), r'$\sqrt{\langle n_b \rangle + \langle n_b \rangle^{2} }$']
]

for idx, line in enumerate(lines[:4]):
    line[0][0].set_label(line[1])
    axes[idx//2, idx%2].legend()
    
print("max_photon_flux_a=phase_matching_bandwidth * max(<ada>)= {:.1e} photons/sec".format(phase_matching_bandwidth * max(expected_vals_dict["na"])))

fig.tight_layout()


### State evolution

Next, we will plot the photon distributions of two quantum optical modes a and b at selected times. The time indices are spaced over Ntsamples time steps and are stored in the t_idx_vec list. 

The following subplots show the state evolution (note the plots have shared horizontal and vertical axes). We represent the photon distributions as bar plots on the subplots. 

In [ ]:
import ipywidgets as widgets
from IPython.display import display

xmin,xmax = 0,5

def update_plot(t):
    fig, axes = plt.subplots(1, 2, figsize=(8, 5))
    psia = ptrace(psi_out.states[t], 0)
    psib = ptrace(psi_out.states[t], 1)
    bar_vals_a = real(psia.diag())
    bar_vals_b = real(psib.diag())
#    max_val = max(bar_vals_a.max(), bar_vals_b.max())
    max_val = max(bar_vals_a[1:].max(), bar_vals_b[1:].max()) # ignore first entry P(0) which is always gonna be large
    axes[0].bar(range(N1), bar_vals_a)
    axes[1].bar(range(N2), bar_vals_b)
    axes[0].set_ylabel('$P(n)$')
    axes[1].set_ylabel('$P(n)$')
    axes[0].set_xlabel('$n_a$')
    axes[1].set_xlabel('$n_b$')
    axes[0].set_yscale('log')
    axes[1].set_yscale('log')
    axes[0].set_ylim(bottom=10**-8, top=bar_vals_a[1]*1.2)
    axes[1].set_ylim(bottom=10**-8, top=bar_vals_a[1]*1.2)
    axes[1].set_xlim(xmin,xmax)
    axes[0].set_xlim(xmin,xmax)

    fig.tight_layout()

t_selector = widgets.FloatSlider(
    min=0,
    max=floor(len(tlist)/2),
    step=1,
    value=0,
    description='Time index:',
    continuous_update=False
)

#display(t_selector)
widgets.interact(update_plot, t=widgets.IntSlider(min=0, max=len(tlist)-1, step=1, value=30))


### Photon number statistics

In [ ]:
ops_dict.update({
  "ad_ad_a_a": a.dag() * a.dag() * a * a,
  "bd_bd_b_b": b.dag() * b.dag() * b * b,
  "ad_a_bd_b": a.dag() * a * b.dag() * b
  #"ad_a_bd_b": a.dag() * b.dag() * a * b

})
for key in ops_dict:
    expected_vals_dict[key] = np.zeros(len(psi_out.states))




for idx, psi in enumerate(psi_out.states):
  for key in ops_dict:
    expected_vals_dict[key][idx] = expect(ops_dict[key], psi)


The following code updates the dictionary "ops_dict" by adding entries for the observables being monitored during the evolution of the system. These observables are the second-order coherence terms of the two harmonic oscillators, represented by the combinations of creation and annihilation operators. With self-evident notation, these are "ad_ad_a_a", "bd_bd_b_b", and "ad_a_bd_b".

The code then initializes the dictionary "expected_vals_dict" to store the expected values of these observables over time. This is done by creating an array of zeros with length equal to the number of times at which the system is evaluated during its evolution.

Finally, the code computes the expected values of the observables by iterating over the states in the output of the mesolve function and using the expect function from Qutip to compute the expected values at each time step. These expected values are stored in the corresponding entries of the "expected_vals_dict" dictionary.

### Second-order coherence functions

Walls and Milburn, page 79: Classical fields satisfy the Cauchy-Schwartz inequality evaluated at the same time $t$ on both detectors, i.e., $$ \langle \psi_{out}(t) | a_1^{\dagger} a_1 a_2^{\dagger}  a_2 | \psi_{out}(t) \rangle \leq \langle (a_1^{\dagger})^{2}  (a_1)^{2}  \rangle_t $$, or equivalently 

$$(g_{12}^{(2)})_{t}^2 \leq (g_{1}^{(2)}g_{2}^{(2)})_{t}$$.

Plotting the correlation functions, we see a strong violation of the Cauchy-Schwartz inequality in the weak-pump limit. In fact, the photon correlations of the squeezed fields produced from a vacuum passing through the parametric amplifier are not just strongly nonclassical but actually saturate the bound on the maximum allowed by quantum mechanics. 

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8,5))
background=1e-30
quantities_dict = {
  "g_aa": [r'$g_{11}^{(2)}(t)$', ((expected_vals_dict["ad_ad_a_a"])/(background**2+expected_vals_dict["na"]**2))], 
  "g_bb": [r'$g_{bb}^{(2)}(t)$', ((expected_vals_dict["bd_bd_b_b"])/(background**2+expected_vals_dict["nb"]**2))],
  "g_ab": [r'$g_{ab}^{(2)}(t)$', ((expected_vals_dict["ad_a_bd_b"])/(background**2+expected_vals_dict["na"]*expected_vals_dict["nb"]))],
  "g_ab_squared": [r'$(g_{ab}^{(2)}(t)))^{2}$', ((expected_vals_dict["ad_a_bd_b"])/(background**2+expected_vals_dict["na"]*expected_vals_dict["nb"]))**2]

}

for idx, (quantity_key, (ylabel, data)) in enumerate(quantities_dict.items()):
    line = axes[idx//2, idx%2].plot(tlist, data, 'rx', linewidth=2)
    axes[idx//2, idx%2].set_xlabel("$t$", fontsize=plot_fontsize)
    axes[idx//2, idx%2].set_xlim(1, 20) # 
    axes[idx//2, idx%2].set_ylabel(ylabel, fontsize=plot_fontsize)
#    axes[idx//2, idx%2].set_yscale("log")


fig.tight_layout()


## Sources and further reading


* [Review Article](https://doi.org/10.1080/00107514.2018.1488463)
* [HC Photonics PPLN GUIDE: Overview](https://www.hcphotonics.com/ppln-guide-overview)
* [quED Manual](https://www.qutools.com/files/quED/quED_manual.pdf)
* Saleh and Teich, *Fundamentals of Photonics*
* [Bocquillon et al, PRA 79 035801 (200p](http://dx.doi.org/10.1103/PhysRevA.79.035801
* [Razavi et al, JPhysB 42 114013 (2009)](http://stacks.iop.org/JPhysB/42/114013)
* [Thorn et al, Am Jo of Phys 72, 1210-1219 (2004)](https://aapt.scitation.org/doi/10.1119/1.1737397)
* quTools HBT manual: https://www.qutools.com/files/quED/quED-HBT/quED-HBT-manual.pdf
* quTools secret HBT manual: https://www.qutools.com/files/quED/worksheets/qutools_HBT.pdf
* [M. Beck, J. Opt. Soc. Am. B 24, 2972-2978 (2007)]https://opg.optica.org/josab/abstract.cfm?URI=josab-24-12-2972
* Loudon, *The Quantum Theory of Light*
* Fox, *Quantum Optics: An Introduction*
* Gerry and Knight, *Introductory Quantum Optics*

## Pre-Lab Questions

Before coming to the lab for the HBT experiment, please complete these pre-lab questions and be ready to *show your work*. If you have questions, ask the instructor before starting the lab.

1. What is $\gtwo(0)$ for a number state with 2 photons? What about with 3 photons? Generalize this to Fock state $\ket{n}$ with n photons and plot $\gtwo(0)$ as a function of n.
1. What is $\gtwo(0)$ for a coherent state $\ket{\alpha}$?
1. *Bonus* Consider the case of a 50-50 beamsplitter with inputs $\ahat$ and $\bhat$ and outputs $\chat$ and $\dhat$. $\ahat$ is in the $\ket{1}$ state while $\bhat$ is in the vacuum state. Find the variance of the coincidences $\chat$ and $\dhat$ where the variance of an operator $\hat{o}$ is given by  $\langle(\Delta\hat{o})^2\rangle = \langle\hat{o}^2\rangle -\langle\hat{o}\rangle^2$.